In [29]:
import os
import json
import sys
import pandas as pd

In [2]:
os.chdir('T://lab_research//RES-Folder-UPOD//Echo_label//E_ResearchData//2_ResearchData')

In [13]:
'''
{projects: [{"name": "ECHO", "id": 77, "cuis": "", "tuis": "", 
    "documents":[
        {
            "id": 1, 
            "name": "doc1", 
            "text": "text1",
            "annotations": [
                {
                    "id": 1,
                    "user": "user1", 
                    "cui": 1, 
                    "start": 1,
                    "end": 2, 
                    "value": "value1",
                    "validated": true,
                    "correct": true,
                    "deleted": false,
                    "alternative": false,
                    "killed": false,
                    "meta_anns": {
                        "Negation":{
                            "name": "Negation",
                            "validated": true,
                            "accuracy": 1.0,
                            "value": "Negated"
                        }
                    }
            ]
    ]}
]}
'''
base_dict = {"projects": [{"name": "ECHO_LABEL", "id": 777, "cuis": "", "tuis": "", "documents":[]}]}

In [7]:
# read in jsonl file put in list of dictionaries
with open('outdb_140423.jsonl', 'r') as f:
    data = f.readlines()
    data = [json.loads(line) for line in data]
data_with_span = [d for d in data if "spans" in d.keys()]

In [21]:
# unique labels
set([s['label'] for d in data_with_span for s in d["spans"]])

{'lv_sys_func_hyperdynamic',
 'lv_sys_func_improved',
 'lv_sys_func_mild',
 'lv_sys_func_moderate',
 'lv_sys_func_normal',
 'lv_sys_func_severe',
 'lv_sys_func_unchanged',
 'lv_sys_func_unknown'}

In [11]:
len(data), len(data_with_span)

(5000, 4966)

In [26]:
base_annotation = { "user": "BVE", 
                    "cui": 1, 
                    "start": 1,
                    "end": 2, 
                    "value": "value1",
                    "validated": True,
                    "correct": True,
                    "deleted": False,
                    "alternative": False,
                    "killed": False
                }

documents = []
for idx, d in enumerate(data_with_span):
    new_doc = {"id": idx,
               "name": f"doc_{idx}", 
               "text": d["text"], 
               "annotations":[]}
    for jdx, span in enumerate(d['spans']):
        base_annotation['id'] = jdx
        base_annotation['cui'] = jdx
        base_annotation['start'] = span['start']
        base_annotation['end'] = span['end']
        base_annotation['value'] = d["text"][span['start']:span['end']]
        base_annotation['meta_anns'] = {"LeftVentricleFunction":{
                                            "name": "LeftVentricleFunction",
                                            "validated": True,
                                            "accuracy": 1.0,
                                            "value": span['label']
                                        }
                                      }
    new_doc["annotations"].append(base_annotation)
    documents.append(new_doc)
base_dict['projects'][0]['documents'] = documents

In [28]:
# write out json file
with open('metacat_outdb_140423.json', 'w') as f:
    json.dump(base_dict, f)

In [30]:
# read in json file with pandas
df = pd.read_json('outdb_140423.jsonl', lines=True)
df.reset_index().rename(columns={'index':'name'})[['name', 'text']].to_csv('outdb_140423_medcat.csv', index=False)